In [1]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
import string
import re
import dateutil.parser
%matplotlib inline
pd.set_option('display.max_colwidth', 100)

## Load data

In [2]:
# Load dataset
twitter_data_all = pd.read_csv('../../data/data_collected_ref_cleaned/2_first_solar_tweets_cleaned_v2.csv')

In [3]:
# twitter_data_all.head()

In [4]:
# twitter_data_all.info()

In [5]:
# datetime = pd.to_datetime(twitter_data['CreatedAt'])

## Delete header rows

In [6]:
index_names = twitter_data_all[ twitter_data_all['CreatedAt'] == 'CreatedAt' ].index
index_names

Int64Index([   492,    989,   1208,   1704,   2196,   2665,   3163,   3664,
              4163,   4540,
            ...
            152624, 153118, 153617, 154117, 154615, 155113, 155609, 156105,
            156600, 157096],
           dtype='int64', length=352)

In [7]:
twitter_data_all.drop(index_names, inplace = True)

In [8]:
twitter_data_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156812 entries, 0 to 157163
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   CreatedAt         156812 non-null  object
 1   TweetId           156812 non-null  object
 2   AuthorId          156812 non-null  object
 3   TweetText         156812 non-null  object
 4   Tweet_punct       156812 non-null  object
 5   Tweet_tokenized   156812 non-null  object
 6   Tweet_nonstop     156812 non-null  object
 7   Tweet_stemmed     156812 non-null  object
 8   Tweet_lemmatized  156812 non-null  object
dtypes: object(9)
memory usage: 12.0+ MB


## Select Columns 

In [9]:
columns = ['CreatedAt',  'TweetId', 'AuthorId', 'TweetText', 'Tweet_stemmed']
twitter_data = twitter_data_all[columns]
twitter_data.head()

,CreatedAt,TweetId,AuthorId,TweetText,Tweet_stemmed
0,2021-04-01T00:50:05.000Z,1377422984751775755,1134049078424162304,First Solar Completes Sale of North American O&amp;M Business to NovaSource $FSLR https://t.co/j...,"['first', 'solar', 'complet', 'sale', 'north', 'american', 'oampm', 'busi', 'novasourc', 'fslr',..."
1,2021-03-31T23:54:00.000Z,1377408871212351489,586116334,$ALYI To Reveal New Significant Investment In Company From Leading Texas Based Private Equity Fu...,"['alyi', 'reveal', 'new', 'signific', 'invest', 'compani', 'lead', 'texa', 'base', 'privat', 'eq..."
2,2021-03-31T23:47:40.000Z,1377407278056230912,1313171678625701889,@SolarPowerEU @renewableH2EU @EU_Commission @Akuo_Energy @enelgreenpower @EdpRenewables @Enapter...,"['solarpowereu', 'renewableheu', 'eucommiss', 'akuoenergi', 'enelgreenpow', 'edprenew', 'enapt',..."
3,2021-03-31T23:31:00.000Z,1377403081005150211,831801503694336000,$FSLR sector move confirmed https://t.co/h08Dwn5CIF,"['fslr', 'sector', 'move', 'confirm', 'httpstcohdwncif']"
4,2021-03-31T23:29:07.000Z,1377402607010902019,1355194932445786112,$FSLR NovaSource Power Services is World's Largest Solar O&amp;M Provider with Acquisition of Fi...,"['fslr', 'novasourc', 'power', 'servic', 'world', 'largest', 'solar', 'oampm', 'provid', 'acquis..."


In [10]:
print(dateutil.parser.isoparse('2008-09-03T20:56:35.450686Z'))

2008-09-03 20:56:35.450686+00:00


In [11]:
# twitter_data.head()
def change_time_type(createdAtAtribute):
    changed_time = dateutil.parser.isoparse(createdAtAtribute)
    return changed_time

In [12]:
# twitter_data['datetest'] = dateutil.parser.isoparse((twitter_data['CreatedAt']))
# twitter_data_date_test = twitter_data['CreatedAt'][0]
# twitter_data_date_test
# print(dateutil.parser.isoparse(twitter_data_date_test))
# # testing = change_time_type(twitter_data['CreatedAt'][0])
# print(twitter_data['CreatedAt'])
# print(change_time_type(twitter_data['CreatedAt'][0]))

In [13]:
twitter_data['created_datetime'] = twitter_data['CreatedAt'].apply(lambda x: change_time_type(x))
print(type(twitter_data['CreatedAt'][0]))

<class 'str'>


<ipython-input-13-7a8640f1f01b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  twitter_data['created_datetime'] = twitter_data['CreatedAt'].apply(lambda x: change_time_type(x))


In [14]:
twitter_data.head()

,CreatedAt,TweetId,AuthorId,TweetText,Tweet_stemmed,created_datetime
0,2021-04-01T00:50:05.000Z,1377422984751775755,1134049078424162304,First Solar Completes Sale of North American O&amp;M Business to NovaSource $FSLR https://t.co/j...,"['first', 'solar', 'complet', 'sale', 'north', 'american', 'oampm', 'busi', 'novasourc', 'fslr',...",2021-04-01 00:50:05+00:00
1,2021-03-31T23:54:00.000Z,1377408871212351489,586116334,$ALYI To Reveal New Significant Investment In Company From Leading Texas Based Private Equity Fu...,"['alyi', 'reveal', 'new', 'signific', 'invest', 'compani', 'lead', 'texa', 'base', 'privat', 'eq...",2021-03-31 23:54:00+00:00
2,2021-03-31T23:47:40.000Z,1377407278056230912,1313171678625701889,@SolarPowerEU @renewableH2EU @EU_Commission @Akuo_Energy @enelgreenpower @EdpRenewables @Enapter...,"['solarpowereu', 'renewableheu', 'eucommiss', 'akuoenergi', 'enelgreenpow', 'edprenew', 'enapt',...",2021-03-31 23:47:40+00:00
3,2021-03-31T23:31:00.000Z,1377403081005150211,831801503694336000,$FSLR sector move confirmed https://t.co/h08Dwn5CIF,"['fslr', 'sector', 'move', 'confirm', 'httpstcohdwncif']",2021-03-31 23:31:00+00:00
4,2021-03-31T23:29:07.000Z,1377402607010902019,1355194932445786112,$FSLR NovaSource Power Services is World's Largest Solar O&amp;M Provider with Acquisition of Fi...,"['fslr', 'novasourc', 'power', 'servic', 'world', 'largest', 'solar', 'oampm', 'provid', 'acquis...",2021-03-31 23:29:07+00:00


In [15]:
twitter_data['day'] = twitter_data['created_datetime'].apply(lambda x: "%d/%d/%d" % (x.month, x.day , x.year))

<ipython-input-15-cac6ce402a0b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  twitter_data['day'] = twitter_data['created_datetime'].apply(lambda x: "%d/%d/%d" % (x.month, x.day , x.year))


In [83]:
twitter_data.head()

,CreatedAt,TweetId,AuthorId,TweetText,Tweet_stemmed,created_datetime,day
0,2021-04-01T00:50:05.000Z,1377422984751775755,1134049078424162304,First Solar Completes Sale of North American O&amp;M Business to NovaSource $FSLR https://t.co/j...,"['first', 'solar', 'complet', 'sale', 'north', 'american', 'oampm', 'busi', 'novasourc', 'fslr',...",2021-04-01 00:50:05+00:00,4/1/2021
1,2021-03-31T23:54:00.000Z,1377408871212351489,586116334,$ALYI To Reveal New Significant Investment In Company From Leading Texas Based Private Equity Fu...,"['alyi', 'reveal', 'new', 'signific', 'invest', 'compani', 'lead', 'texa', 'base', 'privat', 'eq...",2021-03-31 23:54:00+00:00,3/31/2021
2,2021-03-31T23:47:40.000Z,1377407278056230912,1313171678625701889,@SolarPowerEU @renewableH2EU @EU_Commission @Akuo_Energy @enelgreenpower @EdpRenewables @Enapter...,"['solarpowereu', 'renewableheu', 'eucommiss', 'akuoenergi', 'enelgreenpow', 'edprenew', 'enapt',...",2021-03-31 23:47:40+00:00,3/31/2021
3,2021-03-31T23:31:00.000Z,1377403081005150211,831801503694336000,$FSLR sector move confirmed https://t.co/h08Dwn5CIF,"['fslr', 'sector', 'move', 'confirm', 'httpstcohdwncif']",2021-03-31 23:31:00+00:00,3/31/2021
4,2021-03-31T23:29:07.000Z,1377402607010902019,1355194932445786112,$FSLR NovaSource Power Services is World's Largest Solar O&amp;M Provider with Acquisition of Fi...,"['fslr', 'novasourc', 'power', 'servic', 'world', 'largest', 'solar', 'oampm', 'provid', 'acquis...",2021-03-31 23:29:07+00:00,3/31/2021


# Group By Date

https://stackoverflow.com/questions/45281297/group-by-week-in-pandas/45281418

In [84]:
# df.groupby('Name').resample('W-Mon', on='Date').sum().reset_index().sort_values(by='Date')

In [85]:
# twitter_data_grouped = twitter_data.resample('W-Mon', on='created_datetime').sum().reset_index().sort_values(by='created_datetime')

In [86]:
# twitter_data_grouped = twitter_data.resample('W-Mon', on='created_datetime').sum().reset_index()

In [97]:
# twitter_data_grouped.info()

In [99]:
# twitter_data_grouped.describe()

# Analysis

In [119]:
twitter_data.groupby('day').count()

,CreatedAt,TweetId,AuthorId,TweetText,Tweet_stemmed,created_datetime
day,,,,,,
1/1/2011,1,1,1,1,1,1
1/1/2012,11,11,11,11,11,11
1/1/2013,6,6,6,6,6,6
1/1/2014,14,14,14,14,14,14
1/1/2015,8,8,8,8,8,8
...,...,...,...,...,...,...
9/9/2016,36,36,36,36,36,36
9/9/2017,4,4,4,4,4,4
9/9/2018,6,6,6,6,6,6
